In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Siamese(nn.Module):
    def __init__(self, args, data):
        super(Siamese, self).__init__()

        self.args = args
        self.d = self.args.word_dim + int(self.args.use_char_emb) * self.args.char_hidden_size
        self.l = self.args.num_perspective

        # ----- Word Representation Layer -----
        self.char_emb = nn.Embedding(args.char_vocab_size, args.char_dim, padding_idx=0)

        self.word_emb = nn.Embedding(args.word_vocab_size, args.word_dim)
        # initialize word embedding with GloVe
        self.word_emb.weight.data.copy_(data.TEXT.vocab.vectors)
        # no fine-tuning for word vectors
        self.word_emb.weight.requires_grad = False
        self.trainingtype = args.training
        self.char_LSTM = nn.LSTM(
            input_size=self.args.char_dim,
            hidden_size=self.args.char_hidden_size,
            num_layers=1,
            bidirectional=False,
            batch_first=True)

        # ----- Context Representation Layer -----
        self.context_LSTM = nn.LSTM(
            input_size=self.d,
            hidden_size=self.args.hidden_size,
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )
        # ----- Prediction Layer -----
        self.pred_fc1 = nn.Linear(self.args.hidden_size * 4, self.args.hidden_size * 2)
        self.pred_fc2 = nn.Linear(self.args.hidden_size * 2, self.args.class_size)

        self.reset_parameters()

    def reset_parameters(self):
        # ----- Word Representation Layer -----
        nn.init.uniform(self.char_emb.weight, -0.005, 0.005)
        # zero vectors for padding
        self.char_emb.weight.data[0].fill_(0)

        # <unk> vectors is randomly initialized
        nn.init.uniform(self.word_emb.weight.data[0], -0.1, 0.1)

        nn.init.kaiming_normal(self.char_LSTM.weight_ih_l0)
        nn.init.constant(self.char_LSTM.bias_ih_l0, val=0)
        nn.init.orthogonal(self.char_LSTM.weight_hh_l0)
        nn.init.constant(self.char_LSTM.bias_hh_l0, val=0)

        # ----- Context Representation Layer -----
        nn.init.kaiming_normal(self.context_LSTM.weight_ih_l0)
        nn.init.constant(self.context_LSTM.bias_ih_l0, val=0)
        nn.init.orthogonal(self.context_LSTM.weight_hh_l0)
        nn.init.constant(self.context_LSTM.bias_hh_l0, val=0)

        nn.init.kaiming_normal(self.context_LSTM.weight_ih_l0_reverse)
        nn.init.constant(self.context_LSTM.bias_ih_l0_reverse, val=0)
        nn.init.orthogonal(self.context_LSTM.weight_hh_l0_reverse)
        nn.init.constant(self.context_LSTM.bias_hh_l0_reverse, val=0)

        # ----- Prediction Layer ----
        nn.init.uniform(self.pred_fc1.weight, -0.005, 0.005)
        nn.init.constant(self.pred_fc1.bias, val=0)

        nn.init.uniform(self.pred_fc2.weight, -0.005, 0.005)
        nn.init.constant(self.pred_fc2.bias, val=0)

    def dropout(self, v):
        return F.dropout(v, p=self.args.dropout, training=self.training)

    def forward(self, **kwargs):
        p = self.word_emb(kwargs['p'])
        h = self.word_emb(kwargs['h'])

        if self.args.use_char_emb:
            # (batch, seq_len, max_word_len) -> (batch * seq_len, max_word_len)
            seq_len_p = kwargs['char_p'].size(1)
            seq_len_h = kwargs['char_h'].size(1)

            char_p = kwargs['char_p'].view(-1, self.args.max_word_len)
            char_h = kwargs['char_h'].view(-1, self.args.max_word_len)

            # (batch * seq_len, max_word_len, char_dim)-> (1, batch * seq_len, char_hidden_size)
            _, (char_p, _) = self.char_LSTM(self.char_emb(char_p))
            _, (char_h, _) = self.char_LSTM(self.char_emb(char_h))

            # (batch, seq_len, char_hidden_size)
            char_p = char_p.view(-1, seq_len_p, self.args.char_hidden_size)
            char_h = char_h.view(-1, seq_len_h, self.args.char_hidden_size)

            # (batch, seq_len, word_dim + char_hidden_size)
            p = torch.cat([p, char_p], dim=-1)
            h = torch.cat([h, char_h], dim=-1)

        p = self.dropout(p)
        h = self.dropout(h)

        # ----- Context Representation Layer -----
        # (batch, seq_len, hidden_size * 2)
        #self.context_LSTM.flatten_parameters()
        con_p, _ = self.context_LSTM(p)
        con_h, _ = self.context_LSTM(h)
        #print(con_p.shape)
        con_p_fw, con_p_bw = torch.split(con_p, self.args.hidden_size, dim=-1)
        #print(con_p_fw.shape)
        con_h_fw, con_h_bw = torch.split(con_h, self.args.hidden_size, dim=-1)

        con_p = self.dropout(con_p)
        con_h = self.dropout(con_h)
        #print(con_p_fw[:,-1,:].shape)
        # 2 * (2, batch, hidden_size) -> 2 * (batch, hidden_size * 2) -> (batch, hidden_size * 4)
        x = torch.cat(
            [con_p_fw[:,-1,:],con_p_bw[:,0,:],con_h_fw[:,-1,:],con_h_bw[:,0,:]], dim=-1)
        
        #print(x.shape)
        x = self.dropout(x)

        # ----- Prediction Layer -----
        x = F.tanh(self.pred_fc1(x))
        x = self.dropout(x)
        x = self.pred_fc2(x)
        return x


In [42]:
import argparse
import copy
import os
import torch

from torch import nn, optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from time import gmtime, strftime

from model.BIMPM import BIMPM
from model.utils import SNLI, Quora
from test import test


def train(args, data):
    model = (Siamese(args, data))
    if args.gpu > -1:
        model.cuda(args.gpu)

    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adam(parameters, lr=args.learning_rate)
    criterion = nn.CrossEntropyLoss()

    writer = SummaryWriter(log_dir='runs/' + args.model_time)

    model.train()
    loss, last_epoch = 0, -1
    max_dev_acc, max_test_acc = 0, 0

    iterator = data.train_iter
    for i, batch in enumerate(iterator):
        present_epoch = int(iterator.epoch)
        if present_epoch == args.epoch:
            break
        if present_epoch > last_epoch:
            print('epoch:', str(present_epoch + 1))
        last_epoch = present_epoch

        if args.data_type == 'SNLI':
            s1, s2 = 'premise', 'hypothesis'
        else:
            s1, s2 = 'q1', 'q2'

        s1, s2 = getattr(batch, s1), getattr(batch, s2)

        # limit the lengths of input sentences up to max_sent_len
        if args.max_sent_len >= 0:
            if s1.size()[1] > args.max_sent_len:
                s1 = s1[:, :args.max_sent_len]
            if s2.size()[1] > args.max_sent_len:
                s2 = s2[:, :args.max_sent_len]

        kwargs = {'p': s1, 'h': s2}

        if args.use_char_emb:
            char_p = Variable(torch.LongTensor(data.characterize(s1)))
            char_h = Variable(torch.LongTensor(data.characterize(s2)))

            if args.gpu > -1:
                char_p = char_p.cuda(args.gpu)
                char_h = char_h.cuda(args.gpu)

            kwargs['char_p'] = char_p
            kwargs['char_h'] = char_h

        pred = (model(**kwargs))
        
        optimizer.zero_grad()
        batch_loss = criterion(pred, batch.label)
        loss += batch_loss.data[0]
        batch_loss.backward()
        optimizer.step()
        del pred
        del batch_loss
        if (i + 1) % args.print_freq == 0:
            dev_loss, dev_acc = test(model, args, data, mode='dev')
            test_loss, test_acc = test(model, args, data)
            c = (i + 1) // args.print_freq

            writer.add_scalar('loss/train', loss, c)
            writer.add_scalar('loss/dev', dev_loss, c)
            writer.add_scalar('acc/dev', dev_acc, c)
            writer.add_scalar('loss/test', test_loss, c)
            writer.add_scalar('acc/test', test_acc, c)

            print('train loss: '+ str(loss) +' / dev loss: '+ str(dev_loss) + '/ test loss:' + str(test_loss) +
                  ' / dev acc:' + str(dev_acc) + 'test acc:' + str(test_acc))

            if dev_acc > max_dev_acc:
                max_dev_acc = dev_acc
                max_test_acc = test_acc
                best_model = copy.deepcopy(model)

            loss = 0
            model.train()

    writer.close()
    print('max dev acc:'+ str(max_dev_acc) + '/ max test acc: ' + str(max_test_acc))

    return best_model


def main():
    import sys
    sys.argv = ['foo']
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch-size', default=128, type=int)
    parser.add_argument('--char-dim', default=20, type=int)
    parser.add_argument('--char-hidden-size', default=50, type=int)
    parser.add_argument('--data-type', default='Quora', help='available: SNLI or Quora')
    parser.add_argument('--dropout', default=0.1, type=float)
    parser.add_argument('--epoch', default=15, type=int)
    parser.add_argument('--gpu', default=1, type=int)
    parser.add_argument('--hidden-size', default=100, type=int)
    parser.add_argument('--learning-rate', default=0.001, type=float)
    parser.add_argument('--max-sent-len', default=-1, type=int,
                        help='max length of input sentences model can accept, if -1, it accepts any length')
    parser.add_argument('--num-perspective', default=20, type=int)
    parser.add_argument('--print-freq', default=500, type=int)
    parser.add_argument('--use-char-emb', default=False, action='store_true')
    parser.add_argument('--word-dim', default=300, type=int)
    parser.add_argument('--training', default=0, type=int)
    args = parser.parse_args()
    print(args.training)
    if args.data_type == 'SNLI':
        print('loading SNLI data...')
        data = SNLI(args)
    elif args.data_type == 'Quora':
        print('loading Quora data...')
        data = Quora(args)
    else:
        raise NotImplementedError('only SNLI or Quora data is possible')

    setattr(args, 'char_vocab_size', len(data.char_vocab))
    setattr(args, 'word_vocab_size', len(data.TEXT.vocab))
    setattr(args, 'class_size', len(data.LABEL.vocab))
    setattr(args, 'max_word_len', data.max_word_len)
    setattr(args, 'model_time', strftime('%H:%M:%S', gmtime()))

    print('training start!')
    best_model = train(args, data)

    if not os.path.exists('saved_models'):
        os.makedirs('saved_models')
    torch.save(best_model.state_dict(), 'saved_models/BIBPM_'+args.data_type+'_'+args.model_time+'train'+args.training+'.pt')
    print('training finished!')


if __name__ == '__main__':
    main()


0
loading Quora data...
training start!
epoch: 1
train loss: 294.4184825718403 / dev loss: 49.21399363875389/ test loss:49.52327512949705 / dev acc:0.6708999872207642test acc:0.6703999638557434
train loss: 271.91084460914135 / dev loss: 46.47843912243843/ test loss:46.25057019293308 / dev acc:0.6901999711990356test acc:0.6942999958992004
train loss: 262.7715002298355 / dev loss: 44.96004927158356/ test loss:44.83750591427088 / dev acc:0.7053999900817871test acc:0.7096999883651733
train loss: 257.15390276908875 / dev loss: 44.9849448800087/ test loss:44.55301072448492 / dev acc:0.7076999545097351test acc:0.7075999975204468
train loss: 255.72855180501938 / dev loss: 42.68616831302643/ test loss:42.76470547914505 / dev acc:0.7311999797821045test acc:0.7290999889373779
train loss: 249.16862561553717 / dev loss: 43.63723450899124/ test loss:43.500356167554855 / dev acc:0.7131999731063843test acc:0.7105000019073486
epoch: 2
train loss: 244.55566601455212 / dev loss: 41.25997471809387/ test l

train loss: 157.01515647023916 / dev loss: 34.852437406778336/ test loss:34.98790846765041 / dev acc:0.8093999624252319test acc:0.7991999983787537
train loss: 156.33865844458342 / dev loss: 34.412106826901436/ test loss:34.69887565821409 / dev acc:0.805899977684021test acc:0.7981999516487122
train loss: 159.18577799759805 / dev loss: 36.38910350203514/ test loss:36.42680036276579 / dev acc:0.8061999678611755test acc:0.7964999675750732
train loss: 158.72115605324507 / dev loss: 33.5940053910017/ test loss:34.11306235939264 / dev acc:0.809499979019165test acc:0.7965999841690063
epoch: 11
train loss: 143.65073038637638 / dev loss: 36.985296458005905/ test loss:38.1807572171092 / dev acc:0.805899977684021test acc:0.7910999655723572
train loss: 147.9505419060588 / dev loss: 35.81538861989975/ test loss:36.34571927040815 / dev acc:0.805899977684021test acc:0.7978999614715576
train loss: 148.86081743240356 / dev loss: 36.619073897600174/ test loss:36.94346824288368 / dev acc:0.801799952983856

TypeError: Can't convert 'int' object to str implicitly

In [ ]:
rnn = nn.LSTM(10, 20, 1, bidirectional=True)
input = Variable(torch.randn(5, 3, 10))
h0 = Variable(torch.randn(2, 3, 20))
c0 = Variable(torch.randn(2, 3, 20))
output, hn = rnn(input, (h0, c0))

In [22]:
hn

(Variable containing:
 (0 ,.,.) = 
 
 Columns 0 to 8 
    0.0262 -0.0300  0.1385  0.0517 -0.1187 -0.1581  0.0058 -0.0630 -0.0708
   0.0431  0.0000 -0.1865  0.2206  0.1467 -0.2882 -0.0504 -0.3230 -0.0736
   0.1322 -0.0603  0.1803  0.1033 -0.2160 -0.1069 -0.0303  0.1110  0.0982
 
 Columns 9 to 17 
    0.1379 -0.0159 -0.0569  0.1671 -0.1168  0.1358 -0.0823  0.0798  0.0735
   0.1638  0.1187 -0.0314 -0.1135 -0.1871  0.0551  0.1918 -0.1770 -0.1505
  -0.0077  0.1058 -0.0803  0.1280 -0.0647 -0.0149  0.1065 -0.1494  0.1058
 
 Columns 18 to 19 
    0.0836 -0.0075
  -0.0716 -0.1395
   0.1533  0.0331
 
 (1 ,.,.) = 
 
 Columns 0 to 8 
    0.0218  0.1405  0.0528  0.0870  0.2355  0.0391  0.1179 -0.1470  0.1026
  -0.1145 -0.0303 -0.1846 -0.0444 -0.2343  0.0695  0.0592 -0.0462  0.0413
   0.1245  0.1497  0.0318 -0.0747  0.1270  0.1806 -0.0454 -0.1718  0.1462
 
 Columns 9 to 17 
   -0.0491  0.0722  0.0430  0.1969 -0.0574 -0.0376  0.1289 -0.0570  0.0927
  -0.0145  0.0295 -0.0678  0.0056  0.1946  0.1184  0

In [18]:
len(hn)

2